# Analyzing Stimulation Results
This tutorial will use the simulation results from the [finite amplitudes stimulation](1_finite_amp.ipynb) and [activation threshold search](2_activation_threshold.ipynb) tutorials. We will analyze the response of membrane voltage and gating variables to extracellular stimulation.

## Creating the fiber and setting up for simulation
As before, we create fiber, waveform, potentials and stimulation object

In [ ]:
import numpy as np
from scipy.stats import norm
from wmglab_neuron import build_fiber, FiberModel, ScaledStim

# create fiber model
n_sections = 133
fiber = build_fiber(FiberModel.MRG_INTERPOLATION, diameter=5.7, n_sections=n_sections)

# Setup for simulation
waveform = np.concatenate((np.ones(200), -np.ones(200), np.zeros(49600)))

fiber.potentials = norm.pdf(np.linspace(-1, 1, n_sections), 0, 0.05) * 100

time_step = 0.001
time_stop = 50

# Create stimulation object
stimulation = ScaledStim(waveform=waveform, dt=time_step, tstop=time_stop)

## Running activation threshold search

As before, we can simulate the response to a single stimulation pulse

In [ ]:
ap, time = stimulation.run_sim(-1, fiber)
print(f'Number of action potentials detected: {ap}')
print(f'Time of last action potential detection: {time}')

We didn't set the fiber to save any variables, so no membrane voltage or gating variable information was stored

In [ ]:
# checks if the fiber object has the given attribute.
print(hasattr(fiber, 'vm'), hasattr(fiber, 'gating'))

Let's set the fiber to save both the membrane voltage, and the gating variables, and rerun the simulation.

In [ ]:
fiber.set_save_vm()  # save membrane voltage
fiber.set_save_gating()  # save gating variables
ap, time = stimulation.run_sim(-1, fiber)

Now that we have saved membrane voltage and gating variables, let's take a look at them.

In [ ]:
print(fiber.vm)
print(fiber.gating)

We have a neuron `Vector` object for each node of the fiber. Notice that since, by default, fibers are created with passive end nodes, the first and last values are "None".

Let's plot membrane voltage for an end compartment, and center compartment.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(font_scale=1.5, style='whitegrid', palette='colorblind')

plt.figure()
plt.plot(np.array(stimulation.time)[:2000], list(fiber.vm[1])[:2000], label='end node')
plt.plot(np.array(stimulation.time)[:2000], list(fiber.vm[6])[:2000], label='center node')
plt.legend()
plt.xlabel('Time (ms)')
plt.ylabel('$V_m (mV)$')
ax2 = plt.gca().twinx()
ax2.plot(np.array(stimulation.time)[:2000], -waveform[:2000], 'r--', label='Stimulus')
ax2.legend(loc=4)
ax2.grid(False)
plt.ylabel('Normalized stimulation amplitude')
plt.show()

We can plot a heatmap of the voltage across all compartments over time.

In [ ]:
import pandas as pd

data = pd.DataFrame(np.array(fiber.vm[1:-1]))
vrest = fiber[0].e_pas
print('Membrane rest voltage:', vrest)
g = sns.heatmap(
    data,
    cbar_kws={'label': '$V_m (mV)$'},
    cmap='seismic',
    vmax=np.amax(data.values) + vrest,
    vmin=-np.amax(data.values) + vrest,
)
plt.xlim([0, 1000])
plt.ylabel('Segment index')
plt.xlabel('Time (ms)')
tick_locs = np.linspace(0, len(np.array(stimulation.time)[:1000]), 9)
labels = [round(np.array(stimulation.time)[int(ind)], 2) for ind in tick_locs]
g.set_xticks(ticks=tick_locs, labels=labels)
plt.title(
    'Membrane voltage over time.\
          \nRed=depolarized, Blue=hyperpolarized'
)

Running a threshold search will also save our variables. Let's try plotting vm at threshold.

In [ ]:
amp, ap = stimulation.find_threshold(fiber)
print(f'Activation threshold: {amp} mA')

In [ ]:
# plot vm
plt.figure()
plt.plot(np.array(stimulation.time)[:2000], list(fiber.vm[1])[:2000], label='end node')
plt.plot(np.array(stimulation.time)[:2000], list(fiber.vm[6])[:2000], label='center node')
plt.legend()
plt.xlabel('Time (ms)')
plt.ylabel('$V_m (mV)$')
ax2 = plt.gca().twinx()
ax2.plot(np.array(stimulation.time)[:2000], -waveform[:2000], 'r--', label='Stimulus')
ax2.legend(loc=4)
ax2.grid(False)
plt.ylabel('Normalized stimulation amplitude')
plt.show()
# plot heatmap
data = pd.DataFrame(np.array(fiber.vm[1:-1]))
vrest = fiber[0].e_pas
print('Membrane rest voltage:', vrest)
g = sns.heatmap(
    data,
    cbar_kws={'label': '$V_m (mV)$'},
    cmap='seismic',
    vmax=np.amax(data.values) + vrest,
    vmin=-np.amax(data.values) + vrest,
)
plt.xlim([0, 1000])
tick_locs = np.linspace(0, len(np.array(stimulation.time)[:1000]), 9)
labels = [round(np.array(stimulation.time)[int(ind)], 2) for ind in tick_locs]
g.set_xticks(ticks=tick_locs, labels=labels)
plt.ylabel('Segment index')
plt.xlabel('Time (ms)')
plt.title(
    'Membrane voltage over time. \
          \nRed=depolarized, Blue=hyperpolarized'
)

Finally, let's take a look at our gating variables

In [ ]:
plt.figure()
for var in fiber.gating:
    plt.plot(np.array(stimulation.time)[:2000], list(fiber.gating[var][6])[:2000], label=var)
plt.legend()
plt.xlabel('Time step')
plt.ylabel('variable value')
plt.show()